In [814]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
import scipy.io as sc
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental import preprocessing
import time
from IPython.display import clear_output

In [815]:
import math 
cons_4 = np.dot(np.sqrt(0.5),[[i,j]for i in c_4 for j in c_4])
cons_16 = [[i,j]for j in c_16 for i in c_16r]

In [816]:
cons_16 = cons_16/np.sqrt(np.mean(np.abs(cons_16)**2))

In [817]:
c_4 = [1,-1]
c_16 = [3,1,-1,-3]
c_16r = [-3,-1,1,3]

In [818]:
import pandas as pd
def dataset(dataFile, labelFile):
    dataFile = "./communication/" + dataFile
    labelFile = "./communication/" + labelFile
    my_data = sc.loadmat(dataFile)
    my_labels = sc.loadmat(labelFile)
    my_data = my_data['Y']
    X = my_labels['L_S_x'].T
    cons = my_labels['L_Constellations'][0]
    data = table_data(my_data)
    label = assign_labels(X, cons)
    return data, label

def assign_labels(X, cons):
    for i in range(len(cons)):
        if cons[i] == 2:
            X[i] = X[i] + 4
    X = X-1
    return X

def table_data(my_data):
    data = np.dstack([my_data.real.T, my_data.imag.T])
    return data

In [819]:
data1 = "hard"
data1_label = "hard_label"
data, label = dataset(data1, data1_label)
test_data = data.reshape(1000,1,50,2)
test_label = label.reshape(1000,1,50,1)
test_label = tf.cast(test_label, tf.float32)

In [820]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
SIGNAL_SIZE = 2
BlockSize = 50
Lambda = 10

In [821]:
def random_crop(image):
    cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])
    return cropped_image

In [822]:
def normalize(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

In [824]:
x = tf.data.Dataset.from_tensor_slices(test_data)
x.map(normalize)

<MapDataset shapes: (1, 50, 2), types: tf.float32>

In [1106]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(25*2*1, use_bias=False, input_shape=[50,2]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((5, 5, 100)))
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.Flatten())
    model.add(layers.Dense(50))
    model.add(layers.Reshape((50,1)))

    return model

In [1107]:
generator = make_generator_model()
generator.summary()

Model: "sequential_158"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_426 (Dense)            (None, 50, 50)            100       
_________________________________________________________________
batch_normalization_72 (Batc (None, 50, 50)            200       
_________________________________________________________________
leaky_re_lu_108 (LeakyReLU)  (None, 50, 50)            0         
_________________________________________________________________
reshape_72 (Reshape)         (None, 5, 5, 100)         0         
_________________________________________________________________
conv2d_transpose_46 (Conv2DT (None, 5, 5, 128)         320000    
_________________________________________________________________
flatten_41 (Flatten)         (None, 3200)              0         
_________________________________________________________________
dense_427 (Dense)            (None, 50)             

In [827]:
noise = tf.random.normal([1,50,2])
test = generator(noise)
test.shape

TensorShape([1, 50, 1])

In [828]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Reshape((5, 5, 2)))
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[5, 5, 2]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

In [1079]:
def mlp_module():
    model = tf.keras.Sequential([
        layers.Dense(128, input_shape=(1,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(20)
    ])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(0.01),
                  metrics=['accuracy'])
    return model
dnn_real_model = mlp_module()

In [1010]:
discriminator = make_discriminator_model()
discriminator(test)
x = test.numpy().reshape(50,1)
x

array([[-0.11847793],
       [ 0.09952572],
       [ 0.10758559],
       [-0.12238979],
       [ 0.00964986],
       [-0.00551957],
       [ 0.0620527 ],
       [-0.12378856],
       [ 0.43204975],
       [-0.09783804],
       [-0.16064131],
       [-0.0035218 ],
       [-0.09310694],
       [ 0.11383899],
       [ 0.08875256],
       [ 0.01911758],
       [ 0.20968178],
       [-0.14024657],
       [ 0.0629466 ],
       [-0.03180377],
       [ 0.01185387],
       [ 0.11307556],
       [ 0.15024345],
       [ 0.15534666],
       [ 0.02849817],
       [-0.11257623],
       [-0.04099222],
       [ 0.08883237],
       [ 0.2890658 ],
       [-0.37759766],
       [ 0.01476518],
       [ 0.18436468],
       [ 0.03158058],
       [ 0.14132768],
       [ 0.2634802 ],
       [-0.17791732],
       [-0.20337816],
       [ 0.09293018],
       [ 0.26617798],
       [-0.05074031],
       [-0.00703778],
       [-0.00621992],
       [-0.1247348 ],
       [-0.02823274],
       [ 0.00171772],
       [-0

In [832]:
decision = discriminator(test)

In [833]:
test_data[0].shape

(1, 50, 2)

In [834]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [835]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [836]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [838]:
def cal_cycle_loss(real, fake):
    loss1 = tf.reduce_mean(tf.abs(real - fake))
    return LAMBDA * loss1

In [839]:
generator_optimizer = tf.keras.optimizers.Adam(1e-2)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-2)
cycle_optimizer = tf.keras.optimizers.Adam(1e-2)

In [1113]:
checkpoint_path = "./checkpoints/train"
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_path, max_to_keep=5)
if ckpt_manager.latest_checkpoint:
    checkpoint.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [1095]:
EPOCHS = 5
# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [1096]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(noise, label):
    with tf.GradientTape(persistent=True) as tape:
        generated = generator(noise, training=True)
        real_output = discriminator(label, training=True)
        fake_output = discriminator(generated, training=True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        cycle_loss = cal_cycle_loss(label, generated)
        total_loss = cycle_loss + gen_loss

    gradients_of_generator = tape.gradient(total_loss, generator.trainable_variables)
    gradients_of_discriminator = tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [1092]:
def train(test_data, test_label, epochs):
    mlp = mlp_module()
    mlp_label = test_label.numpy().reshape(50000,1)
    for epoch in range(epochs):
        mlp_data = []
        start = time.time()
        for i in range(len(test_label)):
            noise = test_data[i]
            label = test_label[i]
            train_step(noise, label)
            mlp_data.append(generator(noise).numpy())
        mlp_data = np.array(mlp_data).reshape(50000,1)
        mlp.fit(mlp_data, mlp_label)
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
        if (epoch + 1) % 5 == 0:
            ckpt_save_path = ckpt_manager.save()
            print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
        
    
    return mlp

In [1093]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [1094]:
mlp = train(test_data, test_label, EPOCHS)

1563/1563 [==============================] - 2s 1ms/step - loss: 2.8010 - accuracy: 0.1265
Time for epoch 1 is 7.002943992614746 sec
1563/1563 [==============================] - 2s 1ms/step - loss: 2.7576 - accuracy: 0.1285
Time for epoch 2 is 6.864525079727173 sec
1563/1563 [==============================] - 2s 1ms/step - loss: 2.7575 - accuracy: 0.1283
Time for epoch 3 is 6.54262900352478 sec
1563/1563 [==============================] - 2s 1ms/step - loss: 2.7576 - accuracy: 0.1277
Time for epoch 4 is 6.682286977767944 sec
1563/1563 [==============================] - 2s 1ms/step - loss: 2.7576 - accuracy: 0.1278
Time for epoch 5 is 6.584009170532227 sec
Saving checkpoint for epoch 5 at ./checkpoints/train/ckpt-4


In [720]:
cal_cycle_loss(generator(test_data[0]), label[0])

<tf.Tensor: shape=(), dtype=float32, numpy=49.94932>

In [1097]:
x = generator(test_data[0]).numpy().reshape(50,1)

In [1098]:
y = test_label[0].numpy().reshape(50,1)

In [936]:
mlp_data = []
for i in range(len(test_label)):
        noise = test_data[i]
        mlp_data.append(generator(noise).numpy().reshape(50))

In [970]:
mlp_data = np.array(mlp_data).reshape(50000,1)

In [971]:
mlp_label = test_label.numpy().reshape(50000,1)

In [972]:
mlp_label.shape

(50000, 1)

In [1089]:
mlp = mlp_module()

In [1025]:
normalizer = preprocessing.Normalization()

In [1026]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))
dnn_real_model = build_and_compile_model(normalizer)

In [1114]:
x = generator(test_data[0])

In [1121]:
cal_cycle_loss(mlp_data, mlp_label)

<tf.Tensor: shape=(), dtype=float32, numpy=85.40047>

In [1120]:
mlp_data

array([[ 5.549143 ],
       [15.923605 ],
       [14.276071 ],
       ...,
       [18.949871 ],
       [24.051914 ],
       [ 3.8551936]], dtype=float32)